In [1]:
import sys
import os
import pandas as pd

# This is a bit of a hack to allow the notebook to import from the services folder
# It adds the parent directory of 'notebooks' (which is your project root) to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from services.data_service import DataService
from services.indicator_service import IndicatorService

print("Libraries and services imported successfully.")

Libraries and services imported successfully.


/Users/aliefchandrawijaya/trading_bot/venv/lib/python3.11/site-packages/pandas_ta/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [2]:
import configparser

# --- LOAD CONFIGURATION ---
config = configparser.ConfigParser()
config.read('../config.ini') 
api_key = config['exchange']['api_key']
api_secret = config['exchange']['api_secret']

# --- Initialize the service WITH the required API keys ---
data_svc = DataService(api_key=api_key, api_secret=api_secret)

# --- Fetch the maximum available historical data ---
# We can go back several years. Let's start from 2018 to get a rich dataset.
START_DATE = "1 Jan, 2018"
historical_df = data_svc.get_all_historical_data(symbol='BTC/USDT', timeframe='4h', start_date=START_DATE)

if historical_df is not None:
    print("\nSuccessfully fetched all historical data.")
    print(f"Total Candles Fetched: {len(historical_df)}")
    print(f"Data Range: from {historical_df.index.min()} to {historical_df.index.max()}")
    display(historical_df.head())

DataService: Binance.US client initialized and connection successful (Read-Only).
DataService: Fetching all historical klines for BTCUSDT since 1 Jan, 2018...
DataService: Downloaded 13011 total klines.
DataService: All historical data fetched and formatted successfully.

Successfully fetched all historical data.
Total Candles Fetched: 13011
Data Range: from 2019-09-23 08:00:00 to 2025-08-31 16:00:00


,open,high,low,close,volume
timestamp,,,,,
2019-09-23 08:00:00,9930.13,9930.13,9930.13,9930.13,0.001
2019-09-23 12:00:00,9930.13,9930.13,9930.13,9930.13,0.000
2019-09-23 16:00:00,9930.13,9930.13,9930.13,9930.13,0.000
2019-09-23 20:00:00,9930.13,9930.13,9930.13,9930.13,0.000
2019-09-24 00:00:00,9930.13,9930.13,9930.13,9930.13,0.000


In [3]:
indicator_svc = IndicatorService()

# Add all the indicators to our historical data
features_df = indicator_svc.add_all_indicators(historical_df)

if features_df is not None:
    print("Successfully added all indicators.")
    print(f"New data shape: {features_df.shape}")
    print("Columns added:", [col for col in features_df.columns if col not in historical_df.columns])
    print("Data Head with Indicators:")
    display(features_df.head())

IndicatorService: Initialized.
IndicatorService: Calculating the final, optimized suite of indicators...
IndicatorService: Final, optimized indicator suite successfully added.
Successfully added all indicators.
New data shape: (12786, 30)
Columns added: ['ichimoku_senkou_span_a', 'ichimoku_senkou_span_b', 'ichimoku_tenkan_sen', 'ichimoku_kijun_sen', 'ichimoku_chikou_span', 'EMA_21', 'EMA_50', 'SMA_200', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14', 'ADX_14', 'DMP_14', 'DMN_14', 'SQZ_20_2.0_20_1.5', 'SQZ_ON', 'SQZ_OFF', 'SQZ_NO']
Data Head with Indicators:


,open,high,low,close,volume,ichimoku_senkou_span_a,ichimoku_senkou_span_b,ichimoku_tenkan_sen,ichimoku_kijun_sen,ichimoku_chikou_span,...,BBB_20_2.0,BBP_20_2.0,ATRr_14,ADX_14,DMP_14,DMN_14,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-10-26 12:00:00,9218.63,9435.76,9045.92,9188.98,99.655543,8156.0825,8136.85,8865.08,8831.335,9154.99,...,38.711030,0.877206,320.684784,48.068949,50.337892,13.708803,1684.348333,0,1,0
2019-10-26 16:00:00,9194.29,9234.36,8905.35,9003.59,102.548898,8156.0825,8129.61,8895.84,8831.335,9099.04,...,39.892238,0.789875,321.279443,48.158986,46.655811,15.831270,1747.121667,0,1,0
2019-10-26 20:00:00,9003.07,9280.85,9001.99,9231.81,38.275267,8156.0825,8112.96,8963.99,8831.335,9098.08,...,40.615307,0.823234,318.249481,48.305935,44.779151,14.840424,1852.043333,0,1,0
2019-10-27 00:00:00,9232.69,9364.63,9100.19,9132.38,44.661409,8168.7375,8112.96,9310.44,8831.335,9107.98,...,40.711207,0.765000,314.405946,48.559202,43.992316,13.948855,1776.460000,0,1,0
2019-10-27 04:00:00,9127.71,9199.00,9077.85,9143.12,22.325593,8131.4150,8112.96,9450.71,8831.335,9211.77,...,40.397292,0.742647,300.601946,48.693015,42.725888,14.078142,1714.025000,0,1,0


In [4]:
# Create a 'data' directory if it doesn't exist to store our processed datasets
data_dir = '../data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Save the DataFrame to a CSV file
output_path = os.path.join(data_dir, 'btc_usd_h4_features.csv')
features_df.to_csv(output_path)

print(f"Processed data with features saved successfully to: {output_path}")

Processed data with features saved successfully to: ../data/btc_usd_h4_features.csv
